In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. WHP Data Imputation')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. WHP Data Imputation/Output')
cur_date = "030923"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
whp_df = data.frame(read_excel("Input/Wildfire_Hazard_030923.xlsx"))
# making the last 2 cols numeric
whp_df[,5:6] = apply(whp_df[,5:6], 2, as.numeric)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(whp_df[, 5:6], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(whp_df[, 5:6], 2, as.numeric):
“NAs introduced by coercion”


In [2]:
head(whp_df)

,GEO_ID,FIPS,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,37001020100,Alamance,201.00,278233.8,92.23704
2,14000US37001020200,37001020200,Alamance,202.00,278233.8,92.23704
3,14000US37001020300,37001020300,Alamance,203.00,278233.8,92.23704
4,14000US37001020400,37001020400,Alamance,204.00,278233.8,92.23704
5,14000US37001020501,37001020501,Alamance,205.01,278233.8,92.23704
6,14000US37001020502,37001020502,Alamance,205.02,278233.8,92.23704


# Variable & GEO Id Filtering

Typically removing variables and GEO ids that have less than 25% of data prior to imputation. However, I know that we're only missing data for 6 GEO ids, so I won't worry about this.

# Random Forest Imputation
Generating missing values using random forest (RF).

In [3]:
# imputing sociodemographic data using RF
random_forest_imputation = function(preimputed_df){

    # missForest doesn't work on a matrix with 2 variables, so throwing in the census tract but will remove
    imputed_RF_object = missForest(as.matrix(preimputed_df[,4:6]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_values_df = data.frame(cbind(preimputed_df[,1:4], imputed_RF_df[,2:3])) 

    
    return(imputed_values_df)
}

# calling function
imputed_df = random_forest_imputation(whp_df)
head(imputed_df)

,GEO_ID,FIPS,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,37001020100,Alamance,201.00,278233.8,92.23704
2,14000US37001020200,37001020200,Alamance,202.00,278233.8,92.23704
3,14000US37001020300,37001020300,Alamance,203.00,278233.8,92.23704
4,14000US37001020400,37001020400,Alamance,204.00,278233.8,92.23704
5,14000US37001020501,37001020501,Alamance,205.01,278233.8,92.23704
6,14000US37001020502,37001020502,Alamance,205.02,278233.8,92.23704


# Adding in WHP Quintiles

In [4]:
# determining number of rows needed in each quintile
# rounding up since the number of rows isn't divisible by 5
quintile_breaks = dim(imputed_df)[1]/5
quintile_breaks

[1] 439

In [5]:
quintile = c(rep("5", times = quintile_breaks), rep("4", times = quintile_breaks), rep("3", times = quintile_breaks), 
             rep("2", times = quintile_breaks), rep("1", times = quintile_breaks))
                                                   
quintile_percentage = c(rep("0-20%", times = quintile_breaks), rep("21-40%", times = quintile_breaks), 
                        rep("41-60%", times = quintile_breaks), rep("61-80%", times = quintile_breaks), 
                        rep("81-100", times = quintile_breaks))

final_df = imputed_df %>%
    arrange(-Wildfire_Hazard_Potential_Mean) %>%
    mutate(Wildfire_Hazard_Potential_Quintile_Number = quintile,
           Wildfire_Hazard_Potential_Quintile_Percentage = quintile_percentage)

head(final_df)

,GEO_ID,FIPS,County,Census_Tract,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,14000US37141920201,37141920201,Pender,9202.01,596842.6,715.5177,5,0-20%
2,14000US37141920401,37141920401,Pender,9204.01,596842.6,715.5177,5,0-20%
3,14000US37141920402,37141920402,Pender,9204.02,596842.6,715.5177,5,0-20%
4,14000US37141920403,37141920403,Pender,9204.03,596842.6,715.5177,5,0-20%
5,14000US37141920601,37141920601,Pender,9206.01,596842.6,715.5177,5,0-20%
6,14000US37141990100,37141990100,Pender,9901.00,596842.6,715.5177,5,0-20%


In [6]:
# exporting data
write.csv(final_df, paste0(Output,"/", cur_date, "_Imputed_WHP_Data.csv"), row.names = FALSE)